In [8]:
from gensim.models import fasttext as ft
import numpy as np
import os

In [9]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_vectors(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))